In [32]:
import pandas as pd
!ls data/raw/cartchar100.csv
df = pd.read_csv('../data/raw/cartchar1000.csv')

data/raw/cartchar100.csv


In [33]:
df.head()

,Unnamed: 0,iexp,object,scattered_re_0,scattered_im_0,scattered_re_1,scattered_im_1,scattered_re_2,scattered_im_2,scattered_re_3,...,scattered_re_20,scattered_im_20,scattered_re_21,scattered_im_21,scattered_re_22,scattered_im_22,scattered_re_23,scattered_im_23,scattered_re_24,scattered_im_24
0,0,0,bone,-0.643782,0.316667,0.025426,-0.017787,0.150845,-0.049688,-0.184055,...,-0.013652,-0.100498,-0.242569,-0.216713,-0.353820,-0.262461,-0.339697,-0.290052,-0.266548,-0.297528
1,1,1,bone,-0.597452,0.286204,0.220622,0.127706,-0.093921,0.017669,0.021089,...,-0.138134,0.083808,-0.298250,0.405085,-0.190139,0.598509,-0.033506,0.479495,0.185918,0.283237
2,2,2,bone,-0.639331,0.392511,-0.141530,0.050119,0.187525,-0.074493,-0.153520,...,-0.049731,-0.169149,-0.151648,0.054111,-0.162559,-0.224041,-0.230108,-0.339921,-0.207182,-0.397847
3,3,3,bone,-0.535237,0.624544,-0.128732,0.184330,0.094458,-0.187616,-0.020455,...,-0.095486,0.126685,-0.124138,0.038768,-0.206072,-0.259239,-0.096171,-0.411476,0.134996,-0.421764
4,4,4,treasure,-0.370699,0.652235,0.591898,-0.248804,-0.329414,0.174941,0.243570,...,0.037414,-0.600326,-0.558950,-0.123384,-0.169168,-0.842588,-0.954160,-0.681612,0.104195,-0.043494


In [34]:
from sklearn.cluster import KMeans
cluster_model = KMeans(n_clusters=3)
X = df[df.filter(regex=("scattered*")).columns]
X.head()
kmeans = cluster_model.fit(X)

In [38]:
df3 = pd.DataFrame({'labels': kmeans.labels_, 'object': df['object']})
df3

,labels,object
0,2,bone
1,0,bone
2,0,bone
3,1,bone
4,0,treasure
...,...,...
995,0,bone
996,2,treasure
997,1,treasure
998,2,treasure


In [51]:
# create a neural network 
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

X = df[df.filter(regex=("scattered*")).columns].values
y_categories = df['object']
y_unique = y_categories.unique()
num_categories = len(y_unique)
y = np.zeros(y_categories.shape[0], np.int32)
for i in range(num_categories):
    name = y_unique[i]
    y[y_categories == name] = i
    
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=567)
    
model = keras.Sequential()
model.add(keras.layers.Dense(X.shape[1]))
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(num_categories, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
700/700 [==============================] - 0s 145us/sample - loss: 1.1290 - acc: 0.3414
Epoch 2/20
700/700 [==============================] - 0s 37us/sample - loss: 1.0113 - acc: 0.4643
Epoch 3/20
700/700 [==============================] - 0s 36us/sample - loss: 0.9176 - acc: 0.6057
Epoch 4/20
700/700 [==============================] - 0s 80us/sample - loss: 0.8261 - acc: 0.6714
Epoch 5/20
700/700 [==============================] - 0s 43us/sample - loss: 0.7352 - acc: 0.7343
Epoch 6/20
700/700 [==============================] - 0s 38us/sample - loss: 0.6473 - acc: 0.7629
Epoch 7/20
700/700 [==============================] - 0s 41us/sample - loss: 0.5655 - acc: 0.8014
Epoch 8/20
700/700 [==============================] - 0s 34us/sample - loss: 0.4955 - acc: 0.8271
Epoch 9/20
700/700 [==============================] - 0s 37us/sample - loss: 0.4299 - acc: 0.8643
Epoch 10/20
700/700 [==============================] - 0s 37us/sample - loss: 0.3660 - acc: 0.8957
Epoch 11/20
700/70

In [61]:
# predict
probs = model.predict(X_test)
y_predict = np.argmax(probs, axis=1)
num_errors = np.sum(y_predict != y_test)
print(f'number of misclassifications: {num_errors} out of {len(y_test)} ({100*float(num_errors)/len(y_test):.2f} percent)')

number of misclassifications: 14 out of 300 (4.67 percent)


In [59]:
y_predict[:10]

array([0, 2, 2, 2, 0, 0, 2, 0, 2, 0])

In [55]:
y_test[:10]

array([0, 2, 2, 2, 0, 0, 2, 0, 2, 0], dtype=int32)